In [1]:
import pandas as pd
import numpy as np

In [3]:
weather = pd.read_csv('weather_daily_agg.csv')
traffic = pd.read_csv('traffic_daily_agg.csv')

In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AVERAGE_FELT_TEMPERATURE  420 non-null    object 
 1   AVERAGE_ROAD_TEMPERATURE  420 non-null    object 
 2   MINIMUM_FELT_TEMPERATURE  420 non-null    object 
 3   DATE                      418 non-null    object 
 4   MINIMUM_PRECIPITATION     418 non-null    float64
 5   MAXIMUM_HUMIDITY          418 non-null    float64
 6   MAXIMUM_FELT_TEMPERATURE  418 non-null    float64
 7   MAXIMUM_PRECIPITATION     418 non-null    float64
 8   MAXIMUM_TEMPERATURE       418 non-null    float64
 9   AVERAGE_HUMIDITY          418 non-null    float64
 10  AVERAGE_WIND              418 non-null    float64
 11  AVERAGE_DIRECTIONOFWIND   418 non-null    float64
 12  MINIMUM_TEMPERATURE       418 non-null    float64
 13  AVERAGE_TEMPERATURE       418 non-null    float64
 14  MAXIMUM_RO

In [7]:
weather[weather['MAXIMUM_HUMIDITY'].isna()]

,AVERAGE_FELT_TEMPERATURE,AVERAGE_ROAD_TEMPERATURE,MINIMUM_FELT_TEMPERATURE,DATE,MINIMUM_PRECIPITATION,MAXIMUM_HUMIDITY,MAXIMUM_FELT_TEMPERATURE,MAXIMUM_PRECIPITATION,MAXIMUM_TEMPERATURE,AVERAGE_HUMIDITY,...,MINIMUM_TEMPERATURE,AVERAGE_TEMPERATURE,MAXIMUM_ROAD_TEMPERATURE,MINIMUM_HUMIDITY,MINIMUM_DIRECTIONOFWIND,MINIMUM_WIND,MAXIMUM_DIRECTIONOFWIND,MINIMUM_ROAD_TEMPERATURE,MAXIMUM_WIND,AVERAGE_PRECIPITATION
343,https://data.ibb.gov.tr/api/3/action/help_show...,False,Internal Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,https://data.ibb.gov.tr/api/3/action/help_show...,False,Internal Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pd.to_datetime(weather['DATE']).dt.to_period('M').unique()

<PeriodArray>
['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
 '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',     'NaT', '2021-02',
 '2021-03', '2021-04']
Length: 16, dtype: period[M]

In [18]:
pd.to_datetime(traffic['DATE']).dt.to_period('M').unique()

<PeriodArray>
['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
 '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02',
 '2021-03', '2021-04', '2021-05']
Length: 17, dtype: period[M]

In [19]:
import requests
import pandas as pd
import json
from data_scrape import get_table_names, run_sql

In [20]:
def weather_agg(table_id):
    columns = run_sql('SELECT * FROM "TABLE_PLACEHOLDER" LIMIT 10', table_id, True)['id'].tolist()
    col_numeric = ''
    daily_avgs = ''
    not_99 = ''
    for col in columns[5:]:
        col_numeric+= f'"{col}"::numeric(5,2),'
        daily_avgs+= f'{col[:3].replace("E","G")}("{col}")::numeric(5,2) AS "{col}",'
        not_99 += f'"{col}"::numeric(5,2) != -99 AND '
    col_numeric = col_numeric[:-1]     
    daily_avgs = daily_avgs[:-1]
    not_99 = not_99[:-5]
    
    sql_query = f'''
    WITH temp_table AS (
    SELECT DATE_TRUNC('day', "DATE_TIME"::date)::date AS "DATE", {col_numeric}
    FROM "TABLE_PLACEHOLDER" 
    WHERE {not_99}
    )

    SELECT "DATE", {daily_avgs}
    FROM temp_table
    GROUP BY "DATE"
    ORDER BY "DATE"
    '''
    return pd.DataFrame(run_sql(sql_query, table_id))

In [ ]:
weather_agg('8ca0081a-333c-4e86-8ea2-ab8741bbfc95').to_csv('weather_daily_agg.csv', sep=',', mode='a', header=False, index=False)